# ESS Instruments

``beamlime`` is designed and implemented to support live data reduction at ESS.

ESS has various instruments and each of them has different range of computation loads.

Here is the plot of ``number of pixel`` and ``event rate`` ranges.

In [ ]:
import sys

sys.path.append("../../")  # To use ``docs`` as a package.

In [ ]:
from ess_requirements import ESSInstruments

ess_requirements = ESSInstruments()
ess_requirements.show()

There is a set of benchmark results we have collected with dummy workflow in various computing environments.

They are collected with the ``benchmarks`` module in ``tests`` package in the repository.

And the ``benchmarks`` module also has loading/visualization helpers.

Here is a contour performance plot of one of the results.

In [ ]:
from loader import collect_reports, merge_measurements
from docs.about.data import benchmark_results
import json

results = benchmark_results()
results_map = json.loads(results.read_text())
report_map = collect_reports(results_map)
df = merge_measurements(report_map)

In [ ]:
# Flatten required hardware specs into columns.
from environments import BenchmarkEnvironment


def retrieve_total_memory(env: BenchmarkEnvironment) -> float:
    return env.hardware_spec.total_memory.value


def retrieve_cpu_cores(env: BenchmarkEnvironment) -> float:
    return env.hardware_spec.cpu_spec.process_cpu_affinity.value


df["total_memory [GB]"] = df["environment"].apply(retrieve_total_memory)
df["cpu_cores"] = df["environment"].apply(retrieve_cpu_cores)

# Fix column names to have proper units.
df.rename(
    columns={
        "num_pixels": "num_pixels [counts]",
        "num_events": "num_events [counts]",
        "num_frames": "num_frames [counts]",
        "event_rate": "event_rate [counts/s]",
    },
    inplace=True,
)

In [ ]:
import scipp as sc
from scipp.compat.pandas_compat import from_pandas, parse_bracket_header

# Convert to scipp dataset.
ds: sc.Dataset = from_pandas(
    df[df["target-name"] == "mini_prototype"].drop(columns=["environment"]),
    header_parser=parse_bracket_header,
    data_columns="time",
)

# Derive speed from time measurements and number of frames.
ds["speed"] = ds.coords["num_frames"] / ds["time"]

In [ ]:
from calculations import sample_mean_per_bin, sample_variance_per_bin

# Calculate mean and variance per bin.
binned = ds["speed"].group("event_rate", "num_pixels", "cpu_cores")
da = sample_mean_per_bin(binned)
da.variances = sample_variance_per_bin(binned).values

In [ ]:
# Select measurement with 63 CPU cores.
da_63_cores = da["cpu_cores", sc.scalar(63, unit=None)]

# Create a meta string with the selected data.
df_63_cores = df[df["cpu_cores"] == 63].reset_index(drop=True)
df_63_cores_envs: BenchmarkEnvironment = df_63_cores["environment"][0]
meta_64_cores = [
    f"{ds.coords['target-name'][0].value} "
    f"of beamlime @ {df_63_cores_envs.git_commit_id[:7]} ",
    f"on {df_63_cores_envs.hardware_spec.operating_system} "
    f"with {df_63_cores_envs.hardware_spec.total_memory.value} "
    f"[{df_63_cores_envs.hardware_spec.total_memory.unit}] of memory, "
    f"{da_63_cores.coords['cpu_cores'].value} CPU cores",
    f"processing total [{df_63_cores['num_frames [counts]'].min()}, {df_63_cores['num_frames [counts]'].max()}] frames ",
]

In [ ]:
# Draw a contour plot.
from matplotlib import pyplot as plt
from visualize import plot_contourf

fig, ax = plt.subplots(1, 1, figsize=(10, 7))
ctr = plot_contourf(
    da_63_cores,
    x_coord="event_rate",
    y_coord="num_pixels",
    fig=fig,
    ax=ax,
    levels=[2, 4, 8, 14, 32, 64, 128],
    extend="both",
    colors=["gold", "yellow", "orange", "lime", "yellowgreen", "green", "darkgreen"],
    under_color="lightgrey",
    over_color="darkgreen",
)
ess_requirements.plot_boundaries(ax)
ess_requirements.configure_full_scale(ax)

ax.set_title("Beamlime Performance Contour Plot")
ax.annotate("14.00 [frame/s]", (5e4, 9e6), size=10)
ax.text(10**4, 10**8, "\n".join(meta_64_cores))
fig.tight_layout()

## Performance Comparisons

We will compare performances of different memory capacity and number of cpu cores.

Performance differences are calculated with the following function ``difference``.

In [ ]:
def difference(da: sc.DataArray, standard_da: sc.DataArray) -> sc.DataArray:
    """Difference from the standard data array in percent."""

    return sc.scalar(100, unit="%") * (da - standard_da) / standard_da

### Memory

More memory capacity did not make any meaningful performance improvement tendency.

In [ ]:
import plopp as pp

mem_da = ds["speed"].group("total_memory", "num_pixels", "event_rate", "cpu_cores")
mem_da = mem_da["cpu_cores", sc.scalar(6, unit=None)].drop_coords(
    "cpu_cores"
)  # Select 6 CPU cores.
memory_comparison = mem_da.flatten(
    ("num_pixels", "event_rate"), "num_pixels_event_rate"
)
mean_memory_comparison = sample_mean_per_bin(memory_comparison)
mean_memory_comparison.variances = sample_variance_per_bin(memory_comparison).values
x_tick_labels = [
    f"{npx=:.0e}\n{er=:.0e}"
    for er, npx in zip(
        mean_memory_comparison.coords["event_rate"].values,
        mean_memory_comparison.coords["num_pixels"].values,
        strict=True,
    )
]

mean_memory_comparison.coords["label"] = sc.arange(
    "num_pixels_event_rate", len(x_tick_labels)
)
standard_mem_speed = mean_memory_comparison["total_memory", 0]
lines_per_memory = {
    f"{mem.value} [{mem.unit}]": difference(
        mean_memory_comparison["total_memory", imem], standard_mem_speed
    )
    for imem, mem in enumerate(mean_memory_comparison.coords["total_memory"])
}
memory_comparison_line_plot = pp.plot(
    lines_per_memory,
    coords=["label"],
    title="Beamlime Performance Comparison per Memory Size",
    figsize=(24, 4),
    grid=True,
)

for i_line, line_name in zip([0, 2], lines_per_memory.keys(), strict=True):
    memory_comparison_line_plot.ax.lines[i_line].set_label(line_name)

df_mem_comparison = df[df["cpu_cores"] == 6].reset_index(drop=True)
df_mem_comparison_env: BenchmarkEnvironment = df_mem_comparison["environment"][0]
meta_mem_comparison = [
    f"on {df_mem_comparison_env.hardware_spec.operating_system} ",
    f"{6} CPU cores",
    f"processing total [{df_mem_comparison['num_frames [counts]'].min()}, "
    f"{df_mem_comparison['num_frames [counts]'].max()}] frames ",
    "difference=100*(speed-standard_speed)/standard_speed, with standard speed: 67 GB",
]

memory_comparison_line_plot.ax.set_xticks(list(range(len(x_tick_labels))))
memory_comparison_line_plot.ax.set_xticklabels(x_tick_labels)
memory_comparison_line_plot.ax.text(-1, 24, "\n".join(meta_mem_comparison))
memory_comparison_line_plot.ax.legend(loc="lower left", title="Memory Size [GB]")

In [ ]:
memory_comparison_line_plot

### CPU Cores

More CPU cores showed improved performance for most cases, especially bigger number of events.

It was expected due to multi-threaded computing of scipp.

In [ ]:
import plopp as pp

cpu_da = ds["speed"].group("total_memory", "num_pixels", "event_rate", "cpu_cores")
cpu_da = cpu_da["total_memory", sc.scalar(135, unit="GB")].drop_coords(
    "total_memory"
)  # Select 135 GB.
cpu_comparison = cpu_da.flatten(("num_pixels", "event_rate"), "num_pixels_event_rate")
mean_cpu_comparison = sample_mean_per_bin(cpu_comparison)
mean_cpu_comparison.variances = sample_variance_per_bin(cpu_comparison).values
mean_cpu_comparison.coords["label"] = sc.arange(
    "num_pixels_event_rate", len(x_tick_labels)
)

standard_cpu_speed = mean_cpu_comparison["cpu_cores", 0]
lines_per_cpu = {
    f"{ncpu.value}": difference(
        mean_cpu_comparison["cpu_cores", icpu], standard_cpu_speed
    )
    for icpu, ncpu in enumerate(mean_cpu_comparison.coords["cpu_cores"])
}
cpu_comparison_line_plot = pp.plot(
    lines_per_cpu,
    coords=["label"],
    title="Beamlime Performance Comparison per Number of CPU Cores",
    figsize=(24, 4),
    grid=True,
    # norm='log',
)

for i_line, line_name in enumerate(lines_per_cpu.keys()):
    cpu_comparison_line_plot.ax.lines[i_line * 2].set_label(line_name)

df_cpu_comparison = df[df["total_memory [GB]"] == 135].reset_index(drop=True)
df_cpu_comparison_env: BenchmarkEnvironment = df_cpu_comparison["environment"][0]
meta_cpu_comparison = [
    f"on {df_cpu_comparison_env.hardware_spec.operating_system} ",
    f"{6} CPU cores",
    f"processing total [{df_cpu_comparison['num_frames [counts]'].min()}, "
    f"{df_cpu_comparison['num_frames [counts]'].max()}] frames ",
]

cpu_comparison_line_plot.ax.set_xticks(list(range(len(x_tick_labels))))
cpu_comparison_line_plot.ax.set_xticklabels(x_tick_labels)
cpu_comparison_line_plot.ax.text(-1, 900, "\n".join(meta_cpu_comparison))
cpu_comparison_line_plot.ax.legend(loc="lower left", title="CPU Cores")

In [ ]:
cpu_comparison_line_plot